In [115]:
#!pip install alembic

import pandas as pd
import numpy as np

#sqlalchemy 
import sqlalchemy
from sqlalchemy import create_engine, inspect, func, table
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Float, Text, DateTime
from sqlalchemy.ext.declarative import declarative_base


In [2]:
#loading 2017 data
tripfulldf = pd.read_csv("data/bikedata.csv")
tripfulldf.head()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,944732,2618,2015-09-24 17:22:00 UTC,Mezes,83,2015-09-24 18:06:00 UTC,Mezes,83,653,94063,Customer
1,984595,5957,2015-10-25 18:12:00 UTC,Mezes,83,2015-10-25 19:51:00 UTC,Mezes,83,52,nil,Customer
2,984596,5913,2015-10-25 18:13:00 UTC,Mezes,83,2015-10-25 19:51:00 UTC,Mezes,83,121,nil,Customer
3,1129385,6079,2016-03-18 10:33:00 UTC,Mezes,83,2016-03-18 12:14:00 UTC,Mezes,83,208,94070,Customer
4,1030383,5780,2015-12-06 10:52:00 UTC,Mezes,83,2015-12-06 12:28:00 UTC,Mezes,83,44,94064,Customer


In [3]:
#renaming column
tripfulldf = tripfulldf.rename(columns={'duration_sec':'duration',})

#converting start and end date to datetime
tripfulldf['start_date'] = pd.to_datetime(tripfulldf['start_date'])
tripfulldf['end_date'] = pd.to_datetime(tripfulldf['end_date'])
tripfulldf.head()

,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,944732,2618,2015-09-24 17:22:00,Mezes,83,2015-09-24 18:06:00,Mezes,83,653,94063,Customer
1,984595,5957,2015-10-25 18:12:00,Mezes,83,2015-10-25 19:51:00,Mezes,83,52,nil,Customer
2,984596,5913,2015-10-25 18:13:00,Mezes,83,2015-10-25 19:51:00,Mezes,83,121,nil,Customer
3,1129385,6079,2016-03-18 10:33:00,Mezes,83,2016-03-18 12:14:00,Mezes,83,208,94070,Customer
4,1030383,5780,2015-12-06 10:52:00,Mezes,83,2015-12-06 12:28:00,Mezes,83,44,94064,Customer


In [36]:
#filtering data for 2015-09-01
mask = (tripfulldf['start_date'] > '2015-09-01')
tripfulldf = tripfulldf.sort_values(by='start_date')
tripPartialdf = tripfulldf.loc[mask]

#checking earliest and latest date in file
print(tripPartialdf.start_date.min())
print(tripPartialdf.start_date.max())

2015-09-01 00:10:00
2016-08-31 23:32:00


In [38]:
#changing new trip file's date time to match datetime in sqlite table
tripPartialdf['start_date'] = tripPartialdf['start_date'].dt.strftime('%m/%d/%Y %-H:%M')
tripPartialdf['end_date'] = tripPartialdf['end_date'].dt.strftime('%m/%d/%Y %-H:%M')
tripPartialdf.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
576902,913465,746,09/01/2015 0:10,San Francisco Caltrain 2 (330 Townsend),69,09/01/2015 0:23,San Francisco City Hall,58,238,94107,Subscriber
443404,913466,969,09/01/2015 0:15,Clay at Battery,41,09/01/2015 0:31,Washington at Kearny,46,16,94133,Subscriber
584634,913467,233,09/01/2015 0:15,Davis at Jackson,42,09/01/2015 0:19,Commercial at Montgomery,45,534,94111,Subscriber
363972,913468,213,09/01/2015 1:29,Clay at Battery,41,09/01/2015 1:32,Steuart at Market,74,312,94107,Subscriber
812801,913469,574,09/01/2015 1:33,Steuart at Market,74,09/01/2015 1:42,San Francisco Caltrain 2 (330 Townsend),69,279,94107,Subscriber


In [14]:
#collecting new stations from 2017
trip2017df = pd.read_csv("data/2017-fordgobike-tripdata.csv")
stationdf = trip2017df[['start_station_id','start_station_name','start_station_latitude','start_station_longitude']]
stationdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519700 entries, 0 to 519699
Data columns (total 4 columns):
start_station_id           519700 non-null int64
start_station_name         519700 non-null object
start_station_latitude     519700 non-null float64
start_station_longitude    519700 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 15.9+ MB


In [15]:
#keeping uniques stations from df
stationdf = stationdf.drop_duplicates()
stationdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 0 to 430614
Data columns (total 4 columns):
start_station_id           272 non-null int64
start_station_name         272 non-null object
start_station_latitude     272 non-null float64
start_station_longitude    272 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 10.6+ KB


In [116]:

# Create the connection engine
engine = create_engine("sqlite:///data/sf_bike_share.sqlite")

#Base.metadata.create_all(engine)

In [117]:
#declaring auto_base
Base = automap_base()

#reflecting tables in the hawaii.db
Base.prepare(engine, reflect=True)

#gathering tables from DB
Base.classes.keys()

['station', 'status', 'trip', 'trip_start_agg']

In [118]:
#setting tables to variables
Station = Base.classes.station
Status = Base.classes.status
Trip = Base.classes.trip
TripS = Base.classes.trip_start_agg

In [119]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

['station', 'status', 'trip', 'trip_start_agg', 'weather']

In [20]:
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
name TEXT
lat NUMERIC
long NUMERIC
dock_count INTEGER
city TEXT
installation_date TEXT


In [21]:
columns = inspector.get_columns('status')
for column in columns:
    print(column["name"], column["type"])

station_id INTEGER
bikes_available INTEGER
docks_available INTEGER
time TEXT
id INTEGER


In [22]:
columns = inspector.get_columns('trip')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
duration INTEGER
start_date TEXT
start_station_name TEXT
start_station_id INTEGER
end_date TEXT
end_station_name TEXT
end_station_id INTEGER
bike_id INTEGER
subscription_type TEXT
zip_code INTEGER


In [120]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [24]:
#checking the data in table
station_list = session.query(Station).limit(2)
for x in station_list:
    print(x.id)
    print(x.name)
    print(x.lat)
    print(x.long)
    print(x.dock_count)
    print(x.city)
    print(x.installation_date)

2
San Jose Diridon Caltrain Station
37.3297320000
-121.9017820000
27
San Jose
8/6/2013
3
San Jose Civic Center
37.3306980000
-121.8889790000
15
San Jose
8/5/2013


/anaconda3/lib/python3.6/site-packages/sqlalchemy/sql/sqltypes.py:596: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [25]:
#checking the data in DB
trip_list = session.query(Trip).limit(5)
for x in trip_list:
    print(x.id)
    print(x.duration)
    print(x.start_date)
    print(x.start_station_name)
    print(x.start_station_id)

4069
174
8/29/2013 9:08
2nd at South Park
64
4073
1067
8/29/2013 9:24
South Van Ness at Market
66
4074
1131
8/29/2013 9:24
South Van Ness at Market
66
4075
1117
8/29/2013 9:24
South Van Ness at Market
66
4076
1118
8/29/2013 9:25
South Van Ness at Market
66


In [33]:
#checking the data commit to DB
trip_dates = session.query(Trip.start_date).all()
tripdatesdf = pd.DataFrame(trip_dates)
tripdatesdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669959 entries, 0 to 669958
Data columns (total 1 columns):
start_date    669959 non-null object
dtypes: object(1)
memory usage: 5.1+ MB


In [30]:
tripdatesdf.head()

,start_date
0,8/29/2013 9:08
1,8/29/2013 9:24
2,8/29/2013 9:24
3,8/29/2013 9:24
4,8/29/2013 9:25


In [37]:
#grabbing max date from Sqlite trip table
tripdatesdf['start_date'] = pd.to_datetime(tripdatesdf['start_date'])
tripdatesdf.start_date.max()

Timestamp('2015-08-31 23:26:00')

In [39]:
tripPartialdf.head()

,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
576902,913465,746,09/01/2015 0:10,San Francisco Caltrain 2 (330 Townsend),69,09/01/2015 0:23,San Francisco City Hall,58,238,94107,Subscriber
443404,913466,969,09/01/2015 0:15,Clay at Battery,41,09/01/2015 0:31,Washington at Kearny,46,16,94133,Subscriber
584634,913467,233,09/01/2015 0:15,Davis at Jackson,42,09/01/2015 0:19,Commercial at Montgomery,45,534,94111,Subscriber
363972,913468,213,09/01/2015 1:29,Clay at Battery,41,09/01/2015 1:32,Steuart at Market,74,312,94107,Subscriber
812801,913469,574,09/01/2015 1:33,Steuart at Market,74,09/01/2015 1:42,San Francisco Caltrain 2 (330 Townsend),69,279,94107,Subscriber


In [93]:
Base = declarative_base()

In [109]:
# Define a Trip Start Grouped Table class
class TripS(Base):
    __tablename__ = "trip_start_agg"
    
    id = Column(Integer, primary_key=True)
    Date = Column(Text)
    station_id = Column(Integer)
    station_name = Column(String)
    ride = Column(Integer)

/anaconda3/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.TripS, and will be replaced in the string-lookup table.
  item.__name__


InvalidRequestError: Table 'trip_start_agg' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [95]:
#adding New trip_start_agg Table to Sqlite DB
conn = engine.connect()
Base.metadata.create_all(conn)

In [42]:
#appending september 2015 - august 2016 trip data to trip sqlite table

#Retrieving last ID in sqlite table
#lastrow = session.query(Trip.id).order_by(Trip.id.desc()).first()
#print(lastrow[0])
#lastrow = lastrow[0]

#count = 1

for index, row in tripPartialdf.iterrows():
    
    tripadd = Trip(id=row['trip_id'], 
                         duration=row['duration'], 
                         start_date=row["start_date"], 
                         start_station_name=row["start_station_name"],
                         start_station_id=row["start_station_id"],
                         end_date=row["end_date"],
                         end_station_name=row["end_station_name"],
                         end_station_id=row["end_station_id"],
                         bike_id=row["bike_number"],
                         subscription_type=row["subscriber_type"],
                         zip_code=row['zip_code'])
    session.add(tripadd)
    session.commit()
    
    #count = count + 1

In [96]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

['station', 'status', 'trip', 'trip_start_agg', 'weather']

In [44]:
#grouping data by most active start station
data = session.query(Trip.start_date,Trip.start_station_id,Trip.start_station_name,func.count(Trip.start_station_id)).group_by(Trip.start_date,Trip.start_station_id,Trip.start_station_name).all()

In [100]:
#transforming query data into DataFrame
datadf = pd.DataFrame(data, columns=['Date:Hour','Station ID','Station Name','Rides'])
datadf.head()

,Date:Hour,Station ID,Station Name,Rides
0,01/01/2016 0:01,70,San Francisco Caltrain (Townsend at 4th),1
1,01/01/2016 0:21,62,2nd at Folsom,2
2,01/01/2016 10:06,54,Embarcadero at Bryant,1
3,01/01/2016 11:05,76,Market at 4th,1
4,01/01/2016 11:09,74,Steuart at Market,2


In [101]:
from pandas.tseries.offsets import MonthEnd
#converting data to datetime
datadf['Date:Hour'] = pd.to_datetime(datadf["Date:Hour"])

In [102]:
#grouping date to day from hour of day
datadf['Date'] = datadf['Date:Hour'].dt.strftime('%Y/%m/%d')

In [103]:
datadf = datadf[['Date','Date:Hour','Station ID','Station Name','Rides']]
datadf.head()

,Date,Date:Hour,Station ID,Station Name,Rides
0,2016/01/01,2016-01-01 00:01:00,70,San Francisco Caltrain (Townsend at 4th),1
1,2016/01/01,2016-01-01 00:21:00,62,2nd at Folsom,2
2,2016/01/01,2016-01-01 10:06:00,54,Embarcadero at Bryant,1
3,2016/01/01,2016-01-01 11:05:00,76,Market at 4th,1
4,2016/01/01,2016-01-01 11:09:00,74,Steuart at Market,2


In [104]:
datadfGB = datadf.groupby(['Date','Station ID','Station Name'])
datagroupdatestartdf = datadfGB.sum()
datagroupdatestartdf.head()

Rides
Date       Station ID Station Name                            
2013/08/29 2          San Jose Diridon Caltrain Station      5
           3          San Jose Civic Center                  9
           4          Santa Clara at Almaden                 3
           5          Adobe on Almaden                       3
           6          San Pedro Square                       4

In [110]:
#exporting data out to csv then reading it back in to fill empty cells
datagroupdatestartdf.to_csv('output/daily_rides_by_start_station.csv')
datagroupdatestartdf = pd.read_csv('output/daily_rides_by_start_station.csv')
datagroupdatestartdf.head()

,Date,Station ID,Station Name,Rides
0,2013/08/29,2,San Jose Diridon Caltrain Station,5
1,2013/08/29,3,San Jose Civic Center,9
2,2013/08/29,4,Santa Clara at Almaden,3
3,2013/08/29,5,Adobe on Almaden,3
4,2013/08/29,6,San Pedro Square,4


In [106]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

['station', 'status', 'trip', 'trip_start_agg', 'weather']

In [98]:
columns = inspector.get_columns('trip_start_agg')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
Date TEXT
station_id INTEGER
station_name VARCHAR
ride INTEGER


In [112]:
#moving agg data to SQLite table
for index, row in datagroupdatestartdf.iterrows():
    
    tripadd = TripS(Date=row['Date'], 
                    station_id=row["Station ID"], 
                    station_name=row["Station Name"],
                    ride=row["Rides"])
    session.add(tripadd)
    session.commit()

In [121]:
#gathering ****END STATION*** data from DB table
data = session.query(Trip.end_date,Trip.end_station_id,Trip.end_station_name,func.count(Trip.end_station_id)).group_by(Trip.end_date,Trip.end_station_id,Trip.end_station_name).all()

In [123]:
#transforming query data into DataFrame
dataendstationdf = pd.DataFrame(data, columns=['Date:Hour','End Station ID','End Station Name','Rides'])
dataendstationdf.head()

,Date:Hour,End Station ID,End Station Name,Rides
0,01/01/2016 0:19,55,Temporary Transbay Terminal (Howard at Beale),1
1,01/01/2016 0:24,74,Steuart at Market,1
2,01/01/2016 10:13,70,San Francisco Caltrain (Townsend at 4th),1
3,01/01/2016 11:43,49,Spear at Folsom,1
4,01/01/2016 11:52,50,Harry Bridges Plaza (Ferry Building),1


In [125]:
from pandas.tseries.offsets import MonthEnd
#converting data to datetime
dataendstationdf['Date:Hour'] = pd.to_datetime(dataendstationdf["Date:Hour"])

#grouping date to day from hour of day
#creating a new date column without timestamp
dataendstationdf['Date'] = dataendstationdf['Date:Hour'].dt.strftime('%Y/%m/%d')

#renaming columns
dataendstationdf = dataendstationdf[['Date','Date:Hour','End Station ID','End Station Name','Rides']]
dataendstationdf.head()

,Date,Date:Hour,End Station ID,End Station Name,Rides
0,2016/01/01,2016-01-01 00:19:00,55,Temporary Transbay Terminal (Howard at Beale),1
1,2016/01/01,2016-01-01 00:24:00,74,Steuart at Market,1
2,2016/01/01,2016-01-01 10:13:00,70,San Francisco Caltrain (Townsend at 4th),1
3,2016/01/01,2016-01-01 11:43:00,49,Spear at Folsom,1
4,2016/01/01,2016-01-01 11:52:00,50,Harry Bridges Plaza (Ferry Building),1


In [127]:
#grouping columns to date, end station ID, and end station name
dataendstationdfGB = dataendstationdf.groupby(['Date','End Station ID','End Station Name'])
datagroupenddatedf = dataendstationdfGB.sum()
datagroupenddatedf.head()

Rides
Date       End Station ID End Station Name                        
2013/08/29 2              San Jose Diridon Caltrain Station      6
           3              San Jose Civic Center                  5
           4              Santa Clara at Almaden                 3
           5              Adobe on Almaden                       2
           6              San Pedro Square                       6

In [128]:
#exporting data out to csv then reading it back in to fill empty cells
datagroupenddatedf.to_csv('output/daily_rides_by_end_station.csv')
datagroupenddatedf = pd.read_csv('output/daily_rides_by_end_station.csv')
datagroupenddatedf.head()

,Date,End Station ID,End Station Name,Rides
0,2013/08/29,2,San Jose Diridon Caltrain Station,6
1,2013/08/29,3,San Jose Civic Center,5
2,2013/08/29,4,Santa Clara at Almaden,3
3,2013/08/29,5,Adobe on Almaden,2
4,2013/08/29,6,San Pedro Square,6


In [129]:
Base = declarative_base()

# Define a class for TRip End table
class TripE(Base):
    __tablename__ = "trip_end_agg"
    
    id = Column(Integer, primary_key=True)
    Date  = Column(Text)
    end_station_id = Column(Integer)
    end_station_name = Column(String)
    ride = Column(Integer)

In [131]:
#adding New trip_start_agg Table to Sqlite DB
conn = engine.connect()
Base.metadata.create_all(conn)

In [132]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

['station', 'status', 'trip', 'trip_end_agg', 'trip_start_agg', 'weather']

In [133]:
#moving agg data to SQLite table
for index, row in datagroupenddatedf.iterrows():
    
    tripadd = TripE(Date=row['Date'], 
                    end_station_id=row["End Station ID"], 
                    end_station_name=row["End Station Name"],
                    ride=row["Rides"])
    session.add(tripadd)
    session.commit()